In [1]:
import pandas as pd
import numpy as np
import random
import gc
from os import listdir
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout, BatchNormalization
from keras.optimizers import SGD, RMSprop
import pickle
from sklearn.externals import joblib
from sklearn import tree
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cross_validation import train_test_split, cross_val_score

Using Theano backend.


# Note types
- 0: nothing
- 1: step
- 2: hold start
- 3: hold/roll end
- 4: roll start
- M: mine

# Classes
- 0: nothing
- 1: one note
- 2: two notes
- 3: three or four notes
- 4: hold start
- 5: roll start
- 6: mine

# Remember
clf = joblib.load('filename.pkl') 

In [6]:
samples_back_included = 8
num_classes = 7
num_features = 40
num_features_total = (num_features * samples_back_included) + 4
save_files = listdir('data')

def get_features_for_index(beat_features, notes, index):
    if index < 0:
        return [0] * num_features
    return beat_features[index]

def get_class_for_index(notes, index):
    if index < 0:
        return [1, 0, 0, 0, 0, 0, 0]
    row = notes[index][0]
    (steps, holds, rolls, mines) = [row.count(char) for char in ['1', '2', '4', 'M']]
    steps += (holds + rolls)
    return [int(i) for i in [steps == 0 and mines == 0, steps == 1, steps == 2, steps > 2, holds > 0, rolls > 0, mines > 0]]
    
importance_rankings = [48, 24, 12, 16, 6, 8, 3, 4, 2, 1]
def get_beat_importance(index):
    for i in range(len(importance_rankings)):
        if index % importance_rankings[i] == 0:
            return i

def get_features_for_song(key, is_full):
    X = []
    y = []
    if '{0}_beat_features.csv'.format(key) in save_files and '{0}_notes.csv'.format(key) in save_files:
        beat_features_rotated = pd.read_csv('data/{0}_beat_features.csv'.format(key)).values
        notes = pd.read_csv('data/{0}_notes.csv'.format(key), converters={'0': lambda x: str(x)}).values
        beat_features = np.flipud(np.rot90(np.array(beat_features_rotated)))
        num_notes = min(len(notes), len(beat_features))
        for i in range(num_notes):
            row_y = get_class_for_index(notes, i)
            if is_full or (not (row_y == 0 and random.randint(0, 5) != 0)):
                features = [feature for j in range(samples_back_included) for feature in get_features_for_index(beat_features, notes, i - j)]
                features.append(i % 48)
                features.append(get_beat_importance(i))
                features.append(i / 48)
                features.append(num_notes - i / 48)
                X.append(features)
                y.append(row_y)
    return np.array(X), np.array(y)

# Total 243 songs
def build_training_data(is_full, start, end):
    X = []
    y = []
    songs_to_use = pd.read_csv('data/songs_to_use.csv').values
    for song_data in songs_to_use[start:end]:
        song_X, song_y = get_features_for_song(song_data[0], is_full)
        X.extend(song_X)
        y.extend(song_y)
    return np.array(X), np.array(y)

In [7]:
X_train, y_train = build_training_data(True, 0, 194)
y_train = np.array(list(map(lambda one_hot: np.argmax(one_hot), y_train)))

In [8]:
X_test, y_test = build_training_data(True, 194, 243)
y_test = np.array(list(map(lambda one_hot: np.argmax(one_hot), y_test)))

In [ ]:
len(X_train)

534504

In [9]:
gc.collect()

42

In [ ]:
gb_clf_2 = GradientBoostingClassifier(random_state=0, learning_rate=0.12, n_estimators=25, max_depth=7, subsample=0.85, max_features=200, verbose=True)
gb_clf_2.fit(X_train, y_train)
print (gb_clf_2.score(X_train, y_train))
print (gb_clf_2.score(X_test, y_test))
# 0.891653430684

In [ ]:
gb_clf = GradientBoostingClassifier(random_state=0, learning_rate=0.12, n_estimators=25, max_depth=10, subsample=0.85, max_features=200, verbose=True)
gb_clf.fit(X_train, y_train)
print (gb_clf.score(X_train, y_train))
print (gb_clf.score(X_test, y_test))
joblib.dump(gb_clf, 'gb_clf1.pkl')
gc.collect()

In [ ]:
gb_clf = GradientBoostingClassifier(random_state=0, learning_rate=0.12, n_estimators=25, max_depth=20, subsample=0.85, max_features=200, verbose=True)
gb_clf.fit(X_train, y_train)
print (gb_clf.score(X_train, y_train))
print (gb_clf.score(X_test, y_test))
joblib.dump(gb_clf, 'gb_clf2.pkl')
gc.collect()

In [ ]:
rf_clf = RandomForestClassifier(n_estimators = 100, verbose=True)
rf_clf.fit(X_train, y_train)
print (rf_clf.score(X_train, y_train))
print (rf_clf.score(X_test, y_test))
joblib.dump(gb_clf, 'rf_clf1.pkl')
gc.collect()
# 0.87058119132163025

In [ ]:
rf_clf = RandomForestClassifier(n_estimators = 25, max_features=200, verbose=True)
rf_clf.fit(X_train, y_train)
print (rf_clf.score(X_train, y_train))
print (rf_clf.score(X_test, y_test))
joblib.dump(gb_clf, 'rf_clf2.pkl')
gc.collect()
# 0.901291443873 accuracy w/ 100 estimators

In [ ]:
rf_clf = RandomForestClassifier(n_estimators = 25, max_features=200, min_samples_split=4, verbose=True)
rf_clf.fit(X_train, y_train)
print (rf_clf.score(X_train, y_train))
print (rf_clf.score(X_test, y_test))
joblib.dump(gb_clf, 'rf_clf4.pkl')
gc.collect()
# 0.87058119132163025

In [ ]:
rf_clf = RandomForestClassifier(n_estimators = 25, max_features=200, min_samples_leaf=2, verbose=True)
rf_clf.fit(X_train, y_train)
print (rf_clf.score(X_train, y_train))
print (rf_clf.score(X_test, y_test))
joblib.dump(gb_clf, 'rf_clf3.pkl')
gc.collect()
# 0.900850706427

In [ ]:
gc.collect()

In [ ]:
rf_clf = RandomForestClassifier(n_estimators = 25, max_features=200, min_samples_split=8, verbose=True)
rf_clf.fit(X_train, y_train)
print (rf_clf.score(X_train, y_train))
print (rf_clf.score(X_test, y_test))
joblib.dump(gb_clf, 'rf_clf5.pkl')
gc.collect()
# 0.87058119132163025

In [ ]:
sgd_clf = SGDClassifier(loss="log", verbose=True)
sgd_clf.fit(X_train, y_train)
print (sgd_clf.score(X_train, y_train))
print (sgd_clf.score(X_test, y_test))
joblib.dump(gb_clf, 'rf_clf6.pkl')
gc.collect()
# 0.849 accuracy

In [ ]:
model = Sequential()

model.add(Dense(512, input_shape=(324,)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(num_classes))
model.add(BatchNormalization())
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adagrad',
              metrics=['accuracy'])

model.fit(X_train, y_train_2, nb_epoch=50, batch_size=64, verbose=1)
print (model.evaluate(X_test, y_test_2, batch_size=64))
model.save('models/song_class_model.h5')
model = None
gc.collect()

# 0.88877616359785194 accuracy